In [1]:
import sys
import os
from io import BytesIO

root = os.path.dirname(os.path.abspath("."))
sys.path.append(root)

import pandas as pd
import numpy as np

# Loading DataFrame

In [2]:
from sklearn.impute import SimpleImputer
import predikit.data.preprocessors.data_cleansing as dc
from predikit.data.preprocessors._base import MissingValueStrategy

my_data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [np.nan, 23, 25, 50, np.nan, 50],
    "Credit": [700, 400, np.nan, 50, 200, np.nan],
}

df = pd.DataFrame(my_data)

mvp = dc.MissingValuesProcessor(add_indicator=True, verbose=True)

mvp.fit_transform(df, cols=["Age", "Credit"])

df

2023-12-04 17:43:18,787 (data_cleansing.py:272) - WARNING: ! Attention Age - 33% Missing!
2023-12-04 17:43:18,789 (data_cleansing.py:272) - WARNING: ! Attention Credit - 33% Missing!


,Name,Age,Credit,Age_isNA,Credit_isNA
0,John,37.0,700.0,1,0
1,Andrea,23.0,400.0,0,0
2,Rose,25.0,337.5,0,1
3,Linda,50.0,50.0,0,0
4,Peter,37.0,200.0,1,0
5,Meg,50.0,337.5,0,1
